In [1]:
# Re-loads all imports every time the cell is ran. 
%load_ext autoreload
%autoreload 2

from time import time

import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.5f}'.format

from IPython.display import display

# Sklearn tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

# Neural Networks
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers.csv_logs import CSVLogger

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt

import os
import csv

/Users/lauren/mambaforge/envs/pytorch_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class TimeseriesDataset(Dataset):   
    '''
    Custom Dataset subclass. 
    Serves as input to DataLoader to transform X 
      into sequence data using rolling window. 
    DataLoader using this dataset will output batches 
      of `(batch_size, seq_len, n_features)` shape.
    Suitable as an input to RNNs. 
    '''
    def __init__(self, X: np.ndarray, y: np.ndarray, seq_len: int = 1):
        self.X = torch.tensor(X).float()
        self.y = torch.tensor(y).float()
        self.seq_len = seq_len

    def __len__(self):
        return self.X.__len__() - (self.seq_len-1)

    def __getitem__(self, index):
        return (self.X[index:index+self.seq_len], self.y[index+self.seq_len-1])

In [3]:
class ParticleAcceleratorDataModule(pl.LightningDataModule):
    '''
    PyTorch Lighting DataModule subclass:
    https://pytorch-lightning.readthedocs.io/en/latest/datamodules.html

    Serves the purpose of aggregating all data loading 
      and processing work in one place.
    '''
    
    def __init__(self, seq_len = 1, batch_size = 128, num_workers=0):
        super().__init__()
        self.seq_len = seq_len
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.X_train = None
        self.y_train = None
        self.X_val = None
        self.y_val = None
        self.X_test = None
        self.X_test = None
        self.columns = None
        self.preprocessing = None

    def prepare_data(self):
        pass

    def setup(self, stage=None):        
        
        df = pd.read_pickle('pickled_df_ALL.pk').diff()


        X = df[['Fwd1Amp',  'Fwd2Amp', 'CavAmp', 'SpareAmp', 'LP_Amp','RevAmp','Fwd1Phs', 'Fwd2Phs', 'Cavphs', 'SparePhs',  'LP_Phase', 'Rev_Phs']]
        y = df['SCam3_Gauss_yCentroid']
        
        X = X.dropna()
        y = y.dropna()
        self.columns = X.columns


        X_cv, X_test, y_cv, y_test = train_test_split(
            X, y, test_size=0.2, shuffle=False
        )
    
        X_train, X_val, y_train, y_val = train_test_split(
            X_cv, y_cv, test_size=0.25, shuffle=False
        )

        preprocessing = RobustScaler()
        preprocessing.fit(X_train)

        if stage == 'fit' or stage is None:
            self.X_train = preprocessing.transform(X_train)
            self.y_train = y_train.values.reshape((-1, 1))
            self.X_val = preprocessing.transform(X_val)
            self.y_val = y_val.values.reshape((-1, 1))

        if stage == 'test' or stage is None:
            self.X_test = preprocessing.transform(X_test)
            self.y_test = y_test.values.reshape((-1, 1))
        

    def train_dataloader(self):
        train_dataset = TimeseriesDataset(self.X_train, 
                                          self.y_train, 
                                          seq_len=self.seq_len)
        train_loader = DataLoader(train_dataset, 
                                  batch_size = self.batch_size, 
                                  shuffle = False, 
                                  num_workers = self.num_workers)
        
        return train_loader

    def val_dataloader(self):
        val_dataset = TimeseriesDataset(self.X_val, 
                                        self.y_val, 
                                        seq_len=self.seq_len)
        val_loader = DataLoader(val_dataset, 
                                batch_size = self.batch_size, 
                                shuffle = False, 
                                num_workers = self.num_workers)

        return val_loader

    def test_dataloader(self):
        test_dataset = TimeseriesDataset(self.X_test, 
                                         self.y_test, 
                                         seq_len=self.seq_len)
        test_loader = DataLoader(test_dataset, 
                                 batch_size = self.batch_size, 
                                 shuffle = False, 
                                 num_workers = self.num_workers)

        return test_loader

In [4]:
class LSTMRegressor(pl.LightningModule):
    '''
    Standard PyTorch Lightning module:
    https://pytorch-lightning.readthedocs.io/en/latest/lightning_module.html
    '''
    def __init__(self, 
                 n_features, 
                 hidden_size, 
                 seq_len, 
                 batch_size,
                 num_layers, 
                 dropout, 
                 learning_rate,
                 criterion):
        super(LSTMRegressor, self).__init__()
        self.n_features = n_features
        self.hidden_size = hidden_size
        self.seq_len = seq_len
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.criterion = criterion
        self.learning_rate = learning_rate

        self.lstm = nn.LSTM(input_size=n_features, 
                            hidden_size=hidden_size,
                            num_layers=num_layers, 
                            dropout=dropout, 
                            batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        # lstm_out = (batch_size, seq_len, hidden_size)
        lstm_out, _ = self.lstm(x)
        y_pred = self.linear(lstm_out[:,-1])
        return y_pred
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        result = self.criterion(y_hat, y)
        return result

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        result= self.criterion(y_hat, y)
        return result
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        result = self.criterion(y_hat, y)
        return result

In [5]:
'''
All parameters are aggregated in one place.
This is useful for reporting experiment params to experiment tracking software
'''

p = dict(
    seq_len = 24,
    batch_size = 70, 
    criterion = nn.MSELoss(),
    max_epochs = 100,
    n_features = 12,
    hidden_size =24,
    num_layers = 1,
    dropout = 0.2,
    learning_rate = 0.001,
)

In [6]:
seed_everything(1)

csv_logger = CSVLogger('./', name='lstm', version='0'),

trainer = Trainer(
    max_epochs=p['max_epochs'],
    logger=csv_logger,
    progress_bar_refresh_rate=2,
)

model = LSTMRegressor(
    n_features = p['n_features'],
    hidden_size = p['hidden_size'],
    seq_len = p['seq_len'],
    batch_size = p['batch_size'],
    criterion = p['criterion'],
    num_layers = p['num_layers'],
    dropout = p['dropout'],
    learning_rate = p['learning_rate']
)

dm = ParticleAcceleratorDataModule(
    seq_len = p['seq_len'],
    batch_size = p['batch_size']
)

dm.setup()

trainer.fit(model, dm)
trainer.test(model, datamodule=dm)

Global seed set to 1
/Users/lauren/mambaforge/envs/pytorch_env/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=2)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/lauren/mambaforge/envs/pytorch_env/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Users/lauren/mambaforge

/Users/lauren/mambaforge/envs/pytorch_env/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 1
/Users/lauren/mambaforge/envs/pytorch_env/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:  75%|█████████▊   | 96/128 [00:00<00:00, 96.37it/s, loss=43.9, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 1:  75%|█████████▊   | 96/128 [00:01<00:00, 86.23it/s, loss=43.9, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 2:  75%|█████████▊   | 96/128 [00:01<00:00, 88.76it/s, loss=43.8, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 3:  75%|█████████▊   | 96/128 [00:01<00:00, 87.89it/s, loss=43.8, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 4:  75%|█████████▊   | 96/128 [00:01<00:00, 50.01it/s, loss=43.8, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 5:  75%|█████████▊   | 96/128 [00:01<00:00, 82.10it/s, loss=43.7, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 6:  75%|█████████▊   | 96/128 [00:01<00:00, 90.31it/s, loss=43.6, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 7:  75%|█████████▊   | 96/128 [00:01<00:00, 75.09it/s, loss=43.3, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 8:  75%|█████████▊   | 96/128 [00:02<00:00, 32.36it/s, loss=42.5, v_num=0]
Validating: 0it [00:00,

Epoch 25:  75%|█████████   | 96/128 [00:01<00:00, 89.13it/s, loss=36.7, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 26:  75%|█████████   | 96/128 [00:01<00:00, 72.99it/s, loss=36.4, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 27:  75%|█████████   | 96/128 [00:01<00:00, 83.46it/s, loss=36.2, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 28:  75%|█████████   | 96/128 [00:01<00:00, 76.20it/s, loss=35.5, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 29:  75%|█████████   | 96/128 [00:01<00:00, 76.26it/s, loss=35.1, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 30:  75%|█████████   | 96/128 [00:01<00:00, 85.69it/s, loss=34.6, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 31:  75%|█████████   | 96/128 [00:01<00:00, 76.30it/s, loss=34.5, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 32:  75%|█████████   | 96/128 [00:01<00:00, 77.13it/s, loss=33.6, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 33:  75%|█████████   | 96/128 [00:01<00:00, 83.62it/s, loss=33.3, v_num=0]
Validating: 0it [00:00,

Epoch 51:  75%|█████████   | 96/128 [00:01<00:00, 84.53it/s, loss=26.2, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 52:  75%|█████████   | 96/128 [00:01<00:00, 89.58it/s, loss=26.1, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 53:  75%|█████████   | 96/128 [00:01<00:00, 72.82it/s, loss=26.1, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 54:  75%|█████████   | 96/128 [00:01<00:00, 68.86it/s, loss=25.6, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 55:  75%|█████████   | 96/128 [00:02<00:00, 46.81it/s, loss=26.1, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 56:  75%|█████████   | 96/128 [00:01<00:00, 49.15it/s, loss=26.5, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 57:  75%|█████████   | 96/128 [00:01<00:00, 56.81it/s, loss=25.2, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 58:  75%|█████████   | 96/128 [00:02<00:00, 44.24it/s, loss=24.5, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 59:  75%|█████████   | 96/128 [00:02<00:00, 46.96it/s, loss=23.9, v_num=0]
Validating: 0it [00:00,

Epoch 77:  75%|█████████   | 96/128 [00:01<00:00, 71.82it/s, loss=21.2, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 78:  75%|██████████▌   | 96/128 [00:01<00:00, 84.63it/s, loss=21, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 79:  75%|█████████   | 96/128 [00:01<00:00, 83.28it/s, loss=20.4, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 80:  75%|█████████   | 96/128 [00:01<00:00, 85.77it/s, loss=20.2, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 81:  75%|█████████   | 96/128 [00:01<00:00, 84.10it/s, loss=20.3, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 82:  75%|█████████   | 96/128 [00:01<00:00, 95.67it/s, loss=19.5, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 83:  75%|█████████   | 96/128 [00:01<00:00, 93.15it/s, loss=19.4, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 84:  75%|█████████   | 96/128 [00:01<00:00, 91.85it/s, loss=20.5, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 85:  75%|█████████   | 96/128 [00:01<00:00, 69.93it/s, loss=20.5, v_num=0]
Validating: 0it [00:00,

/Users/lauren/mambaforge/envs/pytorch_env/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(



Testing:  88%|████████████████████████████▉    | 28/32 [00:00<00:00, 135.46it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------
Testing: 100%|█████████████████████████████████| 32/32 [00:00<00:00, 138.91it/s]


[{}]

In [7]:
metrics = pd.read_csv('./lstm/0/metrics.csv')
train_loss = metrics[['train_loss', 'step', 'epoch']][~np.isnan(metrics['train_loss'])]
val_loss = metrics[['val_loss', 'epoch']][~np.isnan(metrics['val_loss'])]
test_loss = metrics['test_loss'].iloc[-1]

fig, axes = plt.subplots(1, 2, figsize=(16, 5), dpi=100)
axes[0].set_title('Train loss per batch')
axes[0].plot(train_loss['step'], train_loss['train_loss'])
axes[1].set_title('Validation loss per epoch')
axes[1].plot(val_loss['epoch'], val_loss['val_loss'], color='orange')
plt.show(block = True)

print('MSE:')
print(f"Train loss: {train_loss['train_loss'].iloc[-1]:.3f}")
print(f"Val loss:   {val_loss['val_loss'].iloc[-1]:.3f}")
print(f'Test loss:  {test_loss:.3f}')

FileNotFoundError: [Errno 2] No such file or directory: './lstm/0/metrics.csv'